In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import json

In [3]:
# load csv as dataframe
df = pd.read_csv('../../../paper_output/output_few_shot_dev/results.csv')
# increase all prompt_index by 1
df['prompt_index'] = df['prompt_index'] + 1
# convert GLEU column to float or NaN
df['GLEU'] = pd.to_numeric(df['GLEU'], errors='coerce')
# remove "file" column
df = df.drop(columns=['file'])
df.head()

,model,prompt_type,prompt_index,split,run,TP,FP,FN,Prec,Rec,F0.5,GLEU,prompt_template
0,facebook/opt-iml-max-30b,3-shot,1,jfleg-dev,1,839,192,992,0.8138,0.4582,0.7045,0.498200,Reply with a corrected version of the input se...
1,facebook/opt-iml-max-30b,few-shot,1,jfleg-dev,1,919,264,957,0.7768,0.4899,0.6954,0.506410,Reply with a corrected version of the input se...
2,facebook/opt-iml-max-30b,3-shot,3,jfleg-dev,1,821,211,1005,0.7955,0.4496,0.6895,0.494307,You are an English language teacher. A student...
3,facebook/opt-iml-max-30b,1-shot,3,jfleg-dev,1,710,176,1018,0.8014,0.4109,0.6734,0.470543,You are an English language teacher. A student...
4,facebook/opt-iml-max-30b,1-shot,1,jfleg-dev,1,637,128,1038,0.8327,0.3803,0.6727,0.461363,Reply with a corrected version of the input se...


In [4]:
# testing
dftest = df[(df['split'] == 'wibea-dev')]
dftest = dftest[dftest['prompt_type'].str.contains('few-shot')]
dftest

,model,prompt_type,prompt_index,split,run,TP,FP,FN,Prec,Rec,F0.5,GLEU,prompt_template
173,stabilityai/StableBeluga2,few-shot,6,wibea-dev,1,3783,6430,3772,0.3704,0.5007,0.3907,NaN,### System: Reply with a corrected version of ...
182,command,few-shot,1,wibea-dev,1,3273,5612,4282,0.3684,0.4332,0.3797,NaN,Reply with a corrected version of the input se...
184,facebook/opt-iml-max-30b,few-shot,1,wibea-dev,1,1315,1141,6240,0.5354,0.1741,0.3783,NaN,Reply with a corrected version of the input se...
219,gpt-3.5-turbo-0613,few-shot-chat,6,wibea-dev,1,4017,8480,3538,0.3214,0.5317,0.3490,NaN,system: Reply with a corrected version of the ...
264,Writer/InstructPalmyra-20b,few-shot-instruct-input,1,wibea-dev,1,2845,6480,4710,0.3051,0.3766,0.3171,NaN,"Below is an instruction that describes a task,..."
271,bigscience/bloomz-7b1,few-shot,1,wibea-dev,1,1271,1913,6284,0.3992,0.1682,0.3132,NaN,Reply with a corrected version of the input se...
275,tiiuae/falcon-40b-instruct,few-shot,1,wibea-dev,1,3441,8496,4114,0.2883,0.4555,0.3111,NaN,Reply with a corrected version of the input se...
302,meta-llama/Llama-2-70b-chat-hf,few-shot-in,6,wibea-dev,1,3617,10025,3938,0.2651,0.4788,0.2911,NaN,<s>[INST] <<SYS>> Reply with a corrected versi...
322,google/flan-t5-xxl,few-shot,1,wibea-dev,1,636,333,6919,0.6563,0.0842,0.2782,NaN,Reply with a corrected version of the input se...


In [5]:
# rename prompt_type to 2-shot-Coyne for all rows where 'few-shot' is in prompt_type
df.loc[df['prompt_type'].str.contains('few-shot'), 'prompt_type'] = '2-shot-Coyne'

# set prompt_index to 1 for all rows where '2-shot-Coyne' is in prompt_type
df.loc[df['prompt_type'] == '2-shot-Coyne', 'prompt_index'] = 1

df.head()

,model,prompt_type,prompt_index,split,run,TP,FP,FN,Prec,Rec,F0.5,GLEU,prompt_template
0,facebook/opt-iml-max-30b,3-shot,1,jfleg-dev,1,839,192,992,0.8138,0.4582,0.7045,0.498200,Reply with a corrected version of the input se...
1,facebook/opt-iml-max-30b,2-shot-Coyne,1,jfleg-dev,1,919,264,957,0.7768,0.4899,0.6954,0.506410,Reply with a corrected version of the input se...
2,facebook/opt-iml-max-30b,3-shot,3,jfleg-dev,1,821,211,1005,0.7955,0.4496,0.6895,0.494307,You are an English language teacher. A student...
3,facebook/opt-iml-max-30b,1-shot,3,jfleg-dev,1,710,176,1018,0.8014,0.4109,0.6734,0.470543,You are an English language teacher. A student...
4,facebook/opt-iml-max-30b,1-shot,1,jfleg-dev,1,637,128,1038,0.8327,0.3803,0.6727,0.461363,Reply with a corrected version of the input se...


In [6]:
# counts of model and prompt_type
df.groupby(['model', 'prompt_type', 'prompt_index']).count()

split  run  TP  FP  FN  \
model                      prompt_type prompt_index                           
Writer/InstructPalmyra-20b 1-shot      1                 3    3   3   3   3   
                                       2                 3    3   3   3   3   
                                       3                 3    3   3   3   3   
                           2-shot      1                 3    3   3   3   3   
                                       2                 3    3   3   3   3   
...                                                    ...  ...  ..  ..  ..   
tiiuae/falcon-40b-instruct 3-shot      2                 3    3   3   3   3   
                                       3                 3    3   3   3   3   
                           4-shot      1                 3    3   3   3   3   
                                       2                 3    3   3   3   3   
                                       3                 3    3   3   3   3   

                                                     Prec  Rec  F0.5  GLEU  \
model                      prompt_type prompt_index                          
Writer/InstructPalmyra-20b 1-shot      1                3    3     3     1   
                                       2                3    3     3     1   
                                       3                3    3     3     1   
                           2-shot      1                3    3     3     1   
                                       2                3    3     3     1   
...                                                   ...  ...   ...   ...   
tiiuae/falcon-40b-instruct 3-shot      2                3    3     3     1   
                                       3                3    3     3     1   
                           4-shot      1                3    3     3     1   
                                       2                3    3     3     1   
                                       3                3    3     3     1   

                                                     prompt_template  
model                      prompt_type prompt_index                   
Writer/InstructPalmyra-20b 1-shot      1                           3  
                                       2                           3  
                                       3                           3  
                           2-shot      1                           3  
                                       2                           3  
...                                                              ...  
tiiuae/falcon-40b-instruct 3-shot      2                           3  
                                       3                           3  
                           4-shot      1                           3  
                                       2                           3  
                                       3                           3  

[117 rows x 10 columns]

In [7]:
# create column combining prompt_type and prompt_index
df['prompt_type_index'] = df['prompt_type'] + '_' + df['prompt_index'].astype(str)
df.prompt_type_index.unique()

array(['3-shot_1', '2-shot-Coyne_1', '3-shot_3', '1-shot_3', '1-shot_1',
       '3-shot_2', '2-shot_2', '4-shot_1', '4-shot_3', '2-shot_1',
       '1-shot_2', '4-shot_2', '2-shot_3'], dtype=object)

In [8]:
def count_model_prompt_type_index(df, prompt_type_index):
    # count model and prompt_type_index
    return df[df['prompt_type_index'] == prompt_type_index].groupby(['model', 'prompt_type_index']).count()


In [9]:
count_model_prompt_type_index(df, prompt_type_index='4-shot_2')

,,prompt_type,prompt_index,split,run,TP,FP,FN,Prec,Rec,F0.5,GLEU,prompt_template
model,prompt_type_index,,,,,,,,,,,,
Writer/InstructPalmyra-20b,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
bigscience/bloomz-7b1,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
command,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
facebook/opt-iml-max-30b,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
google/flan-t5-xxl,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
gpt-3.5-turbo-0613,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
meta-llama/Llama-2-70b-chat-hf,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
stabilityai/StableBeluga2,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3
tiiuae/falcon-40b-instruct,4-shot_2,3,3,3,3,3,3,3,3,3,3,1,3


In [10]:
for model in df.model.unique():
    print(model)
    # select rows where 'dev' in split
    dfdev = df[df['split'].str.contains('dev')]
    print(dfdev[dfdev['model'] == model].prompt_type_index.value_counts())

facebook/opt-iml-max-30b
prompt_type_index
3-shot_1          3
2-shot-Coyne_1    3
3-shot_3          3
1-shot_3          3
1-shot_1          3
3-shot_2          3
4-shot_1          3
4-shot_3          3
2-shot_1          3
2-shot_3          3
1-shot_2          3
4-shot_2          3
2-shot_2          3
Name: count, dtype: int64
tiiuae/falcon-40b-instruct
prompt_type_index
2-shot_2          3
1-shot_2          3
4-shot_2          3
4-shot_3          3
4-shot_1          3
3-shot_2          3
2-shot_3          3
2-shot_1          3
3-shot_1          3
3-shot_3          3
1-shot_3          3
1-shot_1          3
2-shot-Coyne_1    3
Name: count, dtype: int64
gpt-3.5-turbo-0613
prompt_type_index
2-shot_2          3
1-shot_2          3
4-shot_2          3
2-shot_3          3
1-shot_3          3
3-shot_2          3
4-shot_3          3
3-shot_3          3
1-shot_1          3
2-shot_1          3
4-shot_1          3
3-shot_1          3
2-shot-Coyne_1    3
Name: count, dtype: int64
google/flan-t5-xx

In [10]:
# save dataframe 
df.to_csv('../few_shot_dev_results.csv', index=False)